# Sistema de Evaluación Versión 1

Evaluar las respuestas de un LLM cuando hay una única "respuesta correcta".

## Configuración
#### Clave de API y las librerías de Python relevantes.

In [5]:
import os
import openai
import sys
from openai import OpenAI
import utils 

from helper import get_openai_api_key
openai.api_key = get_openai_api_key()
client = OpenAI(api_key=openai.api_key) 

In [6]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = client.chat.completions.create( 
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

#### Obtener los productos y categorías relevantes
Esta es la lista de productos y categorías que se encuentran en el catálogo de productos.

In [8]:
products_and_category = utils.get_products_and_category()
products_and_category

{'Computers and Laptops': ['TechPro Ultrabook',
  'BlueWave Gaming Laptop',
  'PowerLite Convertible',
  'TechPro Desktop',
  'BlueWave Chromebook'],
 'Smartphones and Accessories': ['SmartX ProPhone',
  'MobiTech PowerCase',
  'SmartX MiniPhone',
  'MobiTech Wireless Charger',
  'SmartX EarBuds'],
 'Televisions and Home Theater Systems': ['CineView 4K TV',
  'SoundMax Home Theater',
  'CineView 8K TV',
  'SoundMax Soundbar',
  'CineView OLED TV'],
 'Gaming Consoles and Accessories': ['GameSphere X',
  'ProGamer Controller',
  'GameSphere Y',
  'ProGamer Racing Wheel',
  'GameSphere VR Headset'],
 'Audio Equipment': ['AudioPhonic Noise-Canceling Headphones',
  'WaveSound Bluetooth Speaker',
  'AudioPhonic True Wireless Earbuds',
  'WaveSound Soundbar',
  'AudioPhonic Turntable'],
 'Cameras and Camcorders': ['FotoSnap DSLR Camera',
  'ActionCam 4K',
  'FotoSnap Mirrorless Camera',
  'ZoomMaster Camcorder',
  'FotoSnap Instant Camera']}

### Encontrar nombres de productos y categorías relevantes (versión 1)
Esta podría ser la versión que está en producción.

In [9]:
def find_category_and_product_v1(user_input,products_and_category):

    delimiter = "####"
    system_message = f"""Se te proporcionarán consultas de servicio al cliente. \
    La consulta de servicio al cliente estará delimitada por los caracteres {delimiter}.
    Genera como salida una lista de Python de objetos JSON, donde cada objeto tiene el siguiente formato:
        'category': <una de Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    Y
        'products': <una lista de productos que deben encontrarse en los productos permitidos a continuación>


    Donde las categorías y productos deben encontrarse en la consulta de servicio al cliente.
    Si se menciona un producto, debe estar asociado con la categoría correcta en la lista de productos permitidos a continuación.
    Si no se encuentran productos o categorías, genera una lista vacía.
    

    Enumera todos los productos que son relevantes para la consulta del cliente basándote en qué tan cercanamente se relaciona
    con el nombre del producto y la categoría del producto.
    No asumas, a partir del nombre del producto, ninguna característica o atributo como calidad relativa o precio.

    Los productos permitidos se proporcionan en formato JSON.
    Las claves de cada elemento representan la categoría.
    Los valores de cada elemento son una lista de productos que están dentro de esa categoría.
    Productos permitidos: {products_and_category}
    

    """
    
    few_shot_user_1 = """Quiero la computadora más cara."""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

### Evaluar en algunas consultas

In [10]:
customer_msg_0 = f"""¿Qué televisor puedo comprar si tengo un presupuesto limitado?"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)


    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


In [11]:
customer_msg_1 = f"""Necesito un cargador para mi smartphone"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)


    [{'category': 'Smartphones and Accessories', 'products': ['MobiTech PowerCase', 'MobiTech Wireless Charger']}]
    


In [12]:
customer_msg_2 = f"""
¿Qué computadoras tienen?"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

"\n    [{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]"

In [13]:
customer_msg_3 = f"""
háblame del teléfono smartx pro y la cámara fotosnap, la dslr.
Además, ¿qué televisores tienen?"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)


    [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}]
    


### Casos de prueba más difíciles
Identificar consultas encontradas en producción, donde el modelo no funciona como se esperaba.

In [14]:
customer_msg_4 = f"""
háblame del televisor CineView, el 8K, la consola Gamesphere, la X.
Tengo un presupuesto limitado, ¿qué computadoras tienen?"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,
                                                      products_and_category)
print(products_by_category_4)


    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'CineView 8K TV']},
     {'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X']},
     {'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]


### Modificar el prompt para que funcione en los casos de prueba difíciles

In [15]:
def find_category_and_product_v2(user_input,products_and_category):
    """
    Añadido: No generar ningún texto adicional que no esté en formato JSON.
    Añadido un segundo ejemplo (para few-shot prompting) donde el usuario pregunta por 
    la computadora más barata. En ambos ejemplos, la respuesta mostrada 
    es la lista completa de productos solo en formato JSON.
    """
    delimiter = "####"
    system_message = f"""Se te proporcionarán consultas de servicio al cliente. \
    La consulta de servicio al cliente estará delimitada por los caracteres {delimiter}.
    Genera como salida una lista de Python de objetos JSON, donde cada objeto tiene el siguiente formato:
        'category': <una de Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    Y
        'products': <una lista de productos que deben encontrarse en los productos permitidos a continuación>
    No generes ningún texto adicional que no esté en formato JSON.
    No escribas ningún texto explicativo después de generar el JSON solicitado.


    Donde las categorías y productos deben encontrarse en la consulta de servicio al cliente.
    Si se menciona un producto, debe estar asociado con la categoría correcta en la lista de productos permitidos a continuación.
    Si no se encuentran productos o categorías, genera una lista vacía.
    

    Enumera todos los productos que son relevantes para la consulta del cliente basándote en qué tan cercanamente se relaciona
    con el nombre del producto y la categoría del producto.
    No asumas, a partir del nombre del producto, ninguna característica o atributo como calidad relativa o precio.

    Los productos permitidos se proporcionan en formato JSON.
    Las claves de cada elemento representan la categoría.
    Los valores de cada elemento son una lista de productos que están dentro de esa categoría.
    Productos permitidos: {products_and_category}
    

    """
    
    few_shot_user_1 = """Quiero la computadora más cara. ¿Qué me recomiendas?"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    few_shot_user_2 = """Quiero la computadora más barata. ¿Qué me recomiendas?"""
    few_shot_assistant_2 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

### Evaluar el prompt modificado en los casos de prueba difíciles

In [16]:
customer_msg_3 = f"""
háblame del teléfono smartx pro y la cámara fotosnap, la dslr.
Además, ¿qué televisores tienen?"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)


    [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}]
    


### Pruebas de regresión: verificar que el modelo aún funcione en los casos de prueba anteriores
Comprobar que la modificación del modelo para corregir los casos de prueba difíciles no afecte negativamente su rendimiento en los casos de prueba anteriores.

In [18]:
customer_msg_0 = f"""¿Qué televisor puedo comprar si tengo un presupuesto limitado?"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

 
    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
    


### Reunir el conjunto de desarrollo para pruebas automatizadas

In [19]:
msg_ideal_pairs_set = [
    
    # ej 0
    {'customer_msg':"""¿Qué televisor puedo comprar si tengo un presupuesto limitado?""",
     'ideal_answer':{
         'Televisions and Home Theater Systems':set(
             ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']
         )}
    },

    # ej 1
    {'customer_msg':"""Necesito un cargador para mi smartphone""",
     'ideal_answer':{
         'Smartphones and Accessories':set(
             ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']
         )}
    },
    # ej 2
    {'customer_msg':f"""¿Qué computadoras tienen?""",
     'ideal_answer':{
              'Computers and Laptops':set(
                  ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'
                  ])
                   }
    },

    # ej 3
    {'customer_msg':f"""háblame del teléfono smartx pro y \
    la cámara fotosnap, la dslr.\
    Además, ¿qué televisores tienen?""",
     'ideal_answer':{
         'Smartphones and Accessories':set(
             ['SmartX ProPhone']),
         'Cameras and Camcorders':set(
             ['FotoSnap DSLR Camera']),
         'Televisions and Home Theater Systems':set(
             ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
         }
    }, 
    
    # ej 4
    {'customer_msg':"""háblame del televisor CineView, el 8K, la consola Gamesphere, la X.
Tengo un presupuesto limitado, ¿qué computadoras tienen?""",
     'ideal_answer':{
         'Televisions and Home Theater Systems':set(
             ['CineView 8K TV']),
         'Gaming Consoles and Accessories':set(
             ['GameSphere X']),
         'Computers and Laptops':set(
             ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
         }
    },
    
    # ej 5
    {'customer_msg':f"""¿Qué smartphones tienen?""",
     'ideal_answer':{
              'Smartphones and Accessories':set(
                  ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds'
                  ])
                       }
    },
    # ej 6
    {'customer_msg':f"""Tengo un presupuesto limitado. ¿Puedes recomendarme algunos smartphones?""",
     'ideal_answer':{
         'Smartphones and Accessories':set(
             ['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartX ProPhone', 'MobiTech Wireless Charger']
         )}
    },

    # ej 7 # esto generará un subconjunto de la respuesta ideal
    {'customer_msg':f"""¿Qué consolas de videojuegos serían buenas para mi amigo al que le gustan los juegos de carreras?""",
     'ideal_answer':{
         'Gaming Consoles and Accessories':set([
             'GameSphere X',
             'ProGamer Controller',
             'GameSphere Y',
             'ProGamer Racing Wheel',
             'GameSphere VR Headset'
       ])}
    },
    # ej 8
    {'customer_msg':f"""¿Qué podría ser un buen regalo para mi amigo videógrafo?""",
     'ideal_answer': {
         'Cameras and Camcorders':set([
         'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera', 'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
         ])}
    },
    
    # ej 9
    {'customer_msg':f"""Quisiera una máquina del tiempo en un jacuzzi.""",
     'ideal_answer': []
    }
    
]

### Evaluar casos de prueba comparándolos con las respuestas ideales

In [20]:
import json
def eval_response_with_ideal(response,
                             ideal,
                             debug=False):
    
    if debug:
        print("respuesta")
        print(response)
    
    # json.loads() espera comillas dobles, no simples
    json_like_str = response.replace("'",'"')
    
    # convertir en una lista de diccionarios
    l_of_d = json.loads(json_like_str)
    
    # caso especial cuando la respuesta es una lista vacía
    if l_of_d == [] and ideal == []:
        return 1
    
    # en otro caso, si la respuesta está vacía 
    # o la ideal debería estar vacía, hay una discrepancia
    elif l_of_d == [] or ideal == []:
        return 0
    
    correct = 0    
    
    if debug:
        print("l_of_d es")
        print(l_of_d)
    for d in l_of_d:

        cat = d.get('category')
        prod_l = d.get('products')
        if cat and prod_l:
            # convertir lista a conjunto para comparación
            prod_set = set(prod_l)
            # obtener el conjunto ideal de productos
            ideal_cat = ideal.get(cat)
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"no se encontró la categoría {cat} en la respuesta ideal")
                    print(f"ideal: {ideal}")
                continue
                
            if debug:
                print("conjunto_productos\n",prod_set)
                print()
                print("conjunto_productos_ideal\n",prod_set_ideal)

            if prod_set == prod_set_ideal:
                if debug:
                    print("correcto")
                correct +=1
            else:
                print("incorrecto")
                print(f"conjunto_productos: {prod_set}")
                print(f"conjunto_productos_ideal: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("la respuesta es un subconjunto de la respuesta ideal")
                elif prod_set >= prod_set_ideal:
                    print("la respuesta es un superconjunto de la respuesta ideal")

    # contar correctas sobre el número total de elementos en la lista
    pc_correct = correct / len(l_of_d)
        
    return pc_correct

In [21]:
print(f'Mensaje del cliente: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'Respuesta ideal: {msg_ideal_pairs_set[7]["ideal_answer"]}')

Mensaje del cliente: ¿Qué consolas de videojuegos serían buenas para mi amigo al que le gustan los juegos de carreras?
Respuesta ideal: {'Gaming Consoles and Accessories': {'ProGamer Racing Wheel', 'GameSphere VR Headset', 'GameSphere Y', 'GameSphere X', 'ProGamer Controller'}}


In [22]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                      products_and_category)
print(f'Respuesta: {response}')

eval_response_with_ideal(response,
                         msg_ideal_pairs_set[7]["ideal_answer"])

Respuesta: 
    [{'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X', 'ProGamer Controller', 'GameSphere Y', 'ProGamer Racing Wheel', 'GameSphere VR Headset']}]
    


1.0

### Ejecutar evaluación en todos los casos de prueba y calcular la fracción de casos correctos

In [23]:
# Nota, esto no funcionará si alguna de las llamadas a la API excede el tiempo de espera
score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    print(f"ejemplo {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']
    
    # print("Mensaje del cliente",customer_msg)
    # print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                          products_and_category)

    
    # print("productos_por_categoria",products_by_category)
    score = eval_response_with_ideal(response,ideal,debug=False)
    print(f"{i}: {score}")
    score_accum += score
    

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"Fracción de respuestas correctas de un total de {n_examples}: {fraction_correct}")

ejemplo 0
0: 1.0
ejemplo 1
incorrecto
conjunto_productos: {'MobiTech PowerCase', 'MobiTech Wireless Charger'}
conjunto_productos_ideal: {'MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds'}
la respuesta es un subconjunto de la respuesta ideal
1: 0.0
ejemplo 2
2: 1.0
ejemplo 3
3: 1.0
ejemplo 4
incorrecto
conjunto_productos: {'CineView 4K TV', 'CineView 8K TV'}
conjunto_productos_ideal: {'CineView 8K TV'}
la respuesta es un superconjunto de la respuesta ideal
4: 0.6666666666666666
ejemplo 5
5: 1.0
ejemplo 6
6: 1.0
ejemplo 7
7: 1.0
ejemplo 8
8: 1.0
ejemplo 9
9: 1
Fracción de respuestas correctas de un total de 10: 0.8666666666666666
